<a href="https://colab.research.google.com/github/RafsanJany-44/Research-Wake-NREM-REM/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE

def balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)

  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('https://raw.githubusercontent.com/RafsanJany-44/Thesis_Project/main/All_DATA/without_sn_and_Epoch/EEG_HMC.csv')
dataset.head()
target="Sleep Stage"

In [14]:
dataset[target] = dataset[target].replace(['W'],'Wake')
dataset[target] = dataset[target].replace(['N1'],'NREM')
dataset[target] = dataset[target].replace(['N2'],'NREM')
dataset[target] = dataset[target].replace(['N3'],'NREM')
dataset[target] = dataset[target].replace(['R'],'REM')

In [15]:
dataset[target].value_counts()

classes = np.array(sorted(list(set(dataset[target]))))

In [16]:
classes

array(['NREM', 'REM', 'Wake'], dtype='<U4')

In [17]:
X = dataset.loc[:, dataset.columns!=target]
y = dataset[target]

In [18]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=0)

In [19]:
y_train.value_counts()

NREM    48746
Wake    12964
REM     10952
Name: Sleep Stage, dtype: int64

In [20]:
new_dataset=balance(X_train,y_train)

X_train = new_dataset.loc[:, new_dataset.columns!=target]
y_train = new_dataset[target] 

In [21]:
y_train.value_counts()

NREM    48746
Wake    48746
REM     48746
Name: Sleep Stage, dtype: int64

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
N=5000
k_range = range (2000,N+1,100)
scores={}
scores_list = []
for k in tqdm(k_range):
  classifier = RandomForestClassifier(n_estimators=k,random_state=0)
  classifier.fit(X_train, y_train)
  y_pred=classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))



%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best n_estimators:")
best_estimator=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_estimator)

  3%|▎         | 1/31 [40:41<20:20:31, 2441.04s/it]

2000/5000 round completed......................... Accurecy: 0.8773645533543827


  6%|▋         | 2/31 [1:23:51<20:22:15, 2528.80s/it]

2100/5000 round completed......................... Accurecy: 0.8771969040766716


 10%|▉         | 3/31 [2:08:40<20:14:12, 2601.88s/it]

...

2300/5000 round completed......................... Accurecy: 0.8776439688172344


 16%|█▌        | 5/31 [3:44:44<20:05:01, 2780.82s/it]

2400/5000 round completed......................... Accurecy: 0.8776160272709492


 19%|█▉        | 6/31 [4:36:04<20:00:58, 2882.34s/it]

2500/5000 round completed......................... Accurecy: 0.8777277934560899


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
N=1
k_range = range (30,N+1)
scores={}
scores_list = []
for k in tqdm(k_range):
  classifier = RandomForestClassifier(max_depth=k,random_state=0)
  classifier.fit(X_train, y_train)
  y_pred=classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))



%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best Depth:")
best_depth=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_depth)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_all = RandomForestClassifier(n_estimators=450,max_depth=39,random_state=0)
rf_all.fit(X_train, y_train)
y_pred=rf_all.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))